In [85]:
import pickle
pickle_off = open("restaurant_data_new.pickle", 'rb')
area_venues = pickle.load(pickle_off)
area_venues

,Venue Id,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Distance,Venue Category
0,4aa17172f964a520904020e3,Bentall Centre,49.276322,-123.133288,JOEY Burrard,49.282864,-123.123495,0.005595,New American Restaurant
1,4aa6a84cf964a520834a20e3,Mount Pleasant,49.263330,-123.096588,Martini's,49.263088,-123.108217,0.011631,Italian Restaurant
2,4aa6ad08f964a5209c4a20e3,Mount Pleasant,49.263330,-123.096588,Nirvana Indian Restaruant,49.264336,-123.100818,0.004348,Indian Restaurant
3,4aa6f0caf964a5208e4b20e3,Mount Pleasant,49.263330,-123.096588,Congee Noodle House 粥麵館 (Congee Noodle House),49.263029,-123.102105,0.005525,Chinese Restaurant
4,4aa7350bf964a520464c20e3,Grandview-Woodland,49.270559,-123.067942,Havana,49.273685,-123.069480,0.003484,Cuban Restaurant
...,...,...,...,...,...,...,...,...,...
484,5b2ef843b77c77002c371ede,Grandview-Woodland,49.270559,-123.067942,Pepino’s,49.279134,-123.070567,0.008968,Italian Restaurant
485,5b65213e005ac1002c47548d,Davie Village,49.281803,-123.133288,Mumbai Local,49.280910,-123.132244,0.001374,Indian Restaurant
486,5bba5b613149b9002c1e480b,Davie Village,49.281803,-123.133288,Horin Ramen + Sake,49.286252,-123.127441,0.007347,Ramen Restaurant
487,5c1c5efb56c89f002c116191,Victoria-Fraserview,49.218416,-123.073287,Weirdo Cafe,49.225729,-123.066254,0.010146,Restaurant


In [26]:
CLIENT_ID = 'GKHFHCRG2K0F4B0OP4FHLLGKYA4PT2LVWMBL21GISWBJZFW0' # your Foursquare ID
CLIENT_SECRET = 'BLBVBLLKZQQ0L0B10T0MDX1AE1AN0XY1NQAMOF2MM2YIWWFO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GKHFHCRG2K0F4B0OP4FHLLGKYA4PT2LVWMBL21GISWBJZFW0
CLIENT_SECRET:BLBVBLLKZQQ0L0B10T0MDX1AE1AN0XY1NQAMOF2MM2YIWWFO


In [27]:
import requests # library to handle requests
import pandas as pd
import numpy as np
from os import path

In [32]:
venue_id = '55c40e66498e739ed2c87adf' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
    venue_id,
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION)
result = requests.get(url).json()
print(result)
print(result['response']['venue'].keys())
print(result['response']['venue']['price']['tier'])
print(result['response']['venue']['price']['message'])
print(result['response']['venue']['rating'])

{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5eb6d041df2774001bdac612'}, 'response': {}}


KeyError: 'venue'

In [129]:
pickle_off = open("restaurant_rating_data.pickle", 'rb')
previous_data = pickle.load(pickle_off)
pickle_off.close()
old_size = len(previous_data)

for index in area_venues.index:
    if area_venues.loc[index]['Venue Id'] not in previous_data['Id'].values:
        print(area_venues.loc[index]['Venue Id'])
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
            area_venues.loc[index]['Venue Id'],
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION)
        result = requests.get(url).json()
        try:
            if result['response']['venue']['id'] == area_venues.loc[index]['Venue Id']:
                try:
                    price = result['response']['venue']['price']['tier']
                except:
                    price = np.nan
                try:
                    message = result['response']['venue']['price']['message']
                except:
                    message = np.nan
                try:
                    rating = result['response']['venue']['rating']
                except:
                    rating = np.nan

                print(result['response']['venue']['id'])
                print(result['response']['venue']['price']['tier'])
                print(result['response']['venue']['price']['message'])
                print(result['response']['venue']['rating'])

                add_list = [(result['response']['venue']['id'],
                                    price,
                                    message,
                                    float(rating))]
                columns_list = ['Id', 'Price Category', 'Price Message', 'Rating']
                temp = pd.DataFrame(add_list, columns=columns_list)
                previous_data = previous_data.append(temp)
                print(previous_data.shape, ' ' ,previous_data['Id'].nunique())

                print('##########################################################')             
        except:
            if result['meta']['errorType'] == 'quota_exceeded':
                print('++++++++++\nquota_exceeded\n++++++++++')
                break

pickling_on = open("restaurant_rating_data.pickle","wb")
pickle.dump(previous_data, pickling_on)
pickling_on.close()

    

4aa80589f964a520ca4e20e3
4aa80589f964a520ca4e20e3
2
Moderate
8.9
(30, 4)   30
##########################################################
4aa805ccf964a520cd4e20e3
4aa805ccf964a520cd4e20e3
1
Cheap
8.3
(31, 4)   31
##########################################################
4aa815a0f964a520344f20e3
4aa815a0f964a520344f20e3
2
Moderate
8.6
(32, 4)   32
##########################################################
4aa81803f964a520454f20e3
4aa81803f964a520454f20e3
2
Moderate
8.9
(33, 4)   33
##########################################################
4aa81b5ff964a5205d4f20e3
4aa81b5ff964a5205d4f20e3
2
Moderate
8.0
(34, 4)   34
##########################################################
4aa81dcdf964a5206a4f20e3
4aa81dcdf964a5206a4f20e3
2
Moderate
8.0
(35, 4)   35
##########################################################
4aa82bf1f964a520ce4f20e3
4aa82bf1f964a520ce4f20e3
1
Cheap
8.4
(36, 4)   36
##########################################################
4aa82e4bf964a520de4f20e3
4aa82e4bf964a520de4f20

In [131]:
pickle_off = open("restaurant_rating_data.pickle", 'rb')
previous_data_new = pickle.load(pickle_off)
pickle_off.close()

In [137]:
print('{} new data added'.format(len(previous_data_new)-old_size))

50 new data added


In [ ]:
len(previous_data_new)

In [102]:
columns_list = ['Id', 'Price Category', 'Price Message', 'Rating']

In [134]:
previous_data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79 entries, 0 to 0
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              79 non-null     object 
 1   Price Category  79 non-null     object 
 2   Price Message   79 non-null     object 
 3   Rating          79 non-null     float64
dtypes: float64(1), object(3)
memory usage: 3.1+ KB


In [139]:
previous_data_new.head()

,Id,Price Category,Price Message,Rating
0,4aa17172f964a520904020e3,2,Moderate,8.4
0,4aa6a84cf964a520834a20e3,2,Moderate,8.1
0,4aa6ad08f964a5209c4a20e3,2,Moderate,7.6
0,4aa6f0caf964a5208e4b20e3,1,Cheap,7.8
0,4aa7350bf964a520464c20e3,2,Moderate,8.2


In [136]:
previous_data_new['Id'].nunique()

79